# Compare Signatures

In [51]:
import numpy as np
from scipy.stats import linregress
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import warnings
import re

In [36]:
warnings.filterwarnings("ignore")

In [38]:
def CompareSignature(isAAorEUR, signature):
    # Calculate the statistics of using the signatures
    # computes for each of the signatures and each of the datasets
    # Input: dirName - the name of the directory of the training files
    #        isAAorEUR - train an AA signature (=0) or EUR (=1)
    #        signature - the signature learned on the training set
    # Output: stats - the statistics of the signature, with R2, p-value
    # compared to random signatures and p-value compared to shuffled signatures

    if isAAorEUR == 0:
        fileName = 'AA_validation.txt'
    elif isAAorEUR == 1:
        fileName = 'EUR_validation.txt'
    else:
        raise ValueError('isAAorEUR can take the values 0 or 1 only')

    exprData, residuals, geneTissue = LoadData(fileName)

    NUM_RAND = 10000

    stats = np.zeros(3)
    rand_stats = np.zeros((NUM_RAND, 2))
#     print(rand_stats)

    # stats
    stats[0] = RegressSig(exprData, geneTissue, signature, residuals)
    # random signatures statistics
    rand_stats[:, 0] = RunRandSignatures(exprData, geneTissue, signature, residuals, NUM_RAND)[0]
    # shuffled signatures statistics
    rand_stats[:, 1] = RunShuffledSig(exprData, geneTissue, signature, residuals, NUM_RAND)[0]

    stats[1] = np.sum(stats[0] < rand_stats[:, 0]) / NUM_RAND
    stats[2] = np.sum(stats[0] < rand_stats[:, 1]) / NUM_RAND

    return stats


# CreateLassoSignature, SelectSignature

In [48]:
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.utils import resample


def CreateLassoSignature(data, phen, featureNames, SUBSETS=100):
    # CreateLassoSignature uses lasso with preconditioning and bootstrapping to create a signature.
    # Input: data - an MxN matrix, where M is the number of samples and N is the number of features
    #        phen - the dependent variable of size M
    #        featureNames - the names of the features (N-sized list)
    #        SUBSETS - the optional number of subsets to run. default is 100
    # Output: allSigsGenes - a list with the union of all the signature genes. Is used to produce the signature
    #         featuresSets - a list with SUBSETS number of signatures

    if len(featureNames) != data.shape[1]:
        raise ValueError("Number of feature names must match the number of features in data.")

    if SUBSETS < 1:
        raise ValueError("SUBSETS must be a positive integer.")

    allSigsGenes = []
    featuresSets = []

    for subset in range(SUBSETS):
#         print(f"{subset+1},", end="")

        data_subset, phen_subset = resample(data, phen, random_state=subset)

        lasso = LassoCV(cv=5, n_alphas=400)
        lasso.fit(data_subset, phen_subset)

        selected_features = np.nonzero(lasso.coef_)[0]
        selected_feature_names = [featureNames[i] for i in selected_features]

        if selected_features.size > 0:
            featuresSets.append(selected_feature_names)
            allSigsGenes.extend(selected_feature_names)

#     print("\n")

    allSigsGenes = list(set(allSigsGenes))

    return allSigsGenes, featuresSets


def SelectSignature(feature_sets, features, index, cutoff):
    feature_appears = np.zeros((len(features),))
    features_str = [f[0] + f[1] for f in features]

    for subset in range(len(feature_sets)):
        feature_set = feature_sets[subset][index]

        if feature_set:
            feature_set_str = [f[0] + f[1] for f in feature_set]
            idxFeature = [i for i, f in enumerate(features_str) if f in feature_set_str]
            feature_appears[idxFeature] += 1

    feature_appears /= len(feature_sets)
    selected_features = feature_appears >= cutoff
    signature = [features[i] for i, select in enumerate(selected_features) if select]
    feature_appears = feature_appears[selected_features]

    return signature, feature_appears


# LoadData

In [40]:
def LoadData(fileName):
    # LoadData loads the relevant files.
    # Input: fileName - the name of the file to load
    # Output: exprData - the imputed expression data matrix
    #         residuals - the residuals (dependent variable)
    #         geneTissue - the names of the gene-tissue names (feature names)

    with open(fileName, 'r') as fid:
        tissues = fid.readline().strip().split('\t')
        geneNames = fid.readline().strip().split('\t')
        geneTissue = [tissues[i] + geneNames[i] for i in range(1, len(tissues))]

    data = np.loadtxt(fileName, delimiter='\t', skiprows=2)
    residuals = data[:, 0]
    exprData = data[:, 1:]

    return exprData, residuals, geneTissue


# RegressSig

In [41]:
def RegressSig(exprData, geneTissue, signature, residuals):
    # RegressSig - Run dose regression based on the signature
    # Inputs: exprData - the imputed expression data matrix
    #         geneTissue - the names of the gene-tissue names (feature names)
    #         signature - the trained signature
    #         residuals - the residuals (dependent variable)
    # Outputs: regression_stats - the statistics of the regression (R2)
    top_features_idx = [i for i, item in enumerate(geneTissue) if item in signature]


#     features_str = [f[0] + f[1] for f in geneTissue]
#     sigFeatures_str = [f[0] + f[1] for f in signature]
#     top_features_idx = [i for i, f in enumerate(features_str) if f in sigFeatures_str]

    selectedData = exprData[:, top_features_idx]
    nonconst_idx = np.std(selectedData, axis=0) > np.finfo(float).eps
    selectedData = selectedData[:, nonconst_idx]
    selectedData = (selectedData - np.mean(selectedData, axis=0)) / np.std(selectedData, axis=0)

    X = np.concatenate((np.ones((selectedData.shape[0], 1)), selectedData), axis=1)
    y = residuals

    reg = LinearRegression()
    reg.fit(X, y)

    regression_stats = reg.score(X, y)

    return regression_stats


# RunRandSignatures

In [42]:
def RunRandSignatures(exprData, geneTissue, signature, residuals, numRands):
    # RunRandSignatures - Run dose regression based on the signature
    # Inputs: exprData - the imputed expression data matrix
    #         geneTissue - the names of the gene-tissue names (feature names)
    #         signature - the trained signature
    #         residuals - the residuals (dependent variable)
    #         numRands - the number of random regressions to perform
    # Outputs: rand_stats - the statistics of the regression repeats (R2)

    np.random.seed()
    rand_stats = np.zeros((numRands, 1))
    np.warnings.filterwarnings('ignore', category=np.RankWarning)
    top_features_idx = [i for i, item in enumerate(geneTissue) if item in signature]
#     print(top_features_idx)
    total_indices=np.arange(0,len(geneTissue))
    remaining_features_idx=new_array = total_indices[~np.in1d(total_indices, top_features_idx)]
#     print(remaining_features_idx)

#     features_str = [f[0] + f[1] for f in geneTissue]
#     sigFeatures_str = [f[0] + f[1] for f in signature]
#     relFeatures_str = [f[0] + f[1] for f in geneTissue]
#     top_features_idx = np.in1d(features_str, sigFeatures_str)
#     rel_features_idx = np.in1d(features_str, relFeatures_str)
#     remaining_features_idx = np.where(top_features_idx == False)
#     print(top_features_idx)
#     print(remaining_features_idx.shape)
#     remaining_features_idx = np.where((rel_features_idx - top_features_idx) > 0)[0]

    if np.sum(top_features_idx) > 0:
        for r in range(numRands):
            p = np.random.permutation(len(remaining_features_idx))
            rand_features_idx = remaining_features_idx[p[:len(signature)]]

            test_data = exprData[:, rand_features_idx]
            nonconst_idx = np.std(test_data, axis=0) > np.finfo(float).eps
            scaler = StandardScaler()
            test_data = scaler.fit_transform(test_data)
#             test_data = (test_data - np.mean(test_data, axis=0)) / np.std(test_data, axis=0)

            X = np.concatenate((np.ones((test_data.shape[0], 1)), test_data[:, nonconst_idx]), axis=1)
            y = residuals

            reg = LinearRegression()
            reg.fit(X, y)

            rand_stats[r, 0] = reg.score(X, y)

    np.warnings.filterwarnings('default', category=np.RankWarning)
#     print(rand_stats)
    return rand_stats


# RunShuffledSig

In [43]:
def RunShuffledSig(exprData, geneTissue, signature, residuals, numRands):
    # RunShuffledSig - Run dose regression based on the signature
    # Inputs: exprData - the imputed expression data matrix
    #         geneTissue - the names of the gene-tissue names (feature names)
    #         signature - the trained signature
    #         residuals - the residuals (dependent variable)
    #         numRands - the number of shuffled regressions to perform
    # Outputs: rand_stats - the statistics of the regression repeats (R2)

    np.random.seed()
    rand_stats = np.zeros((numRands, 1))
    np.warnings.filterwarnings('ignore', category=np.RankWarning)

    features_str = [f[0] + f[1] for f in geneTissue]
    sigFeatures_str = [f[0] + f[1] for f in signature]
    top_features_idx = np.in1d(features_str, sigFeatures_str)

    if np.sum(top_features_idx) == 0:
        return rand_stats

    test_data = exprData[:, top_features_idx]
    nonconst_idx = np.std(test_data, axis=0) > np.finfo(float).eps
    scaler = StandardScaler()
    test_data = scaler.fit_transform(test_data)
    
#     test_data = (test_data - np.mean(test_data, axis=0)) / np.std(test_data, axis=0)

    for r in range(numRands):
        p = np.random.permutation(exprData.shape[0])
        rand_residuals = residuals[p]

        X = np.concatenate((np.ones((test_data.shape[0], 1)), test_data[:, nonconst_idx]), axis=1)
        y = rand_residuals

        reg = LinearRegression()
        reg.fit(X, y)

        rand_stats[r, 0] = reg.score(X, y)

    np.warnings.filterwarnings('default', category=np.RankWarning)
    return rand_stats


# SelectRobustSignature

In [44]:
def SelectRobustSignature(signatureSets, selectedFeatures, cutoff):
    # SelectRobustSignature selects a robust signature from multiple LASSO runs.
    # Inputs: signatureSets - the set of signatures returned from multiple runs
    #         selectedFeatures - the selectedFeatures in any of the runs
    #         cutoff - the minimal percentage of signatures a gene-tissue needs
    #                  to be in, in order to be considered robust
    # Output: signature - a cell array with the robust signature
    feature_appears={}
    for subset in signatureSets:
        for feature in subset:
            if feature in feature_appears:
                feature_appears[feature]+=1
            else:
                feature_appears[feature]=1
    feature_appears = {key: value / len(signatureSets) for key, value in feature_appears.items()}
    signature = [key for key, value in feature_appears.items() if value > cutoff]
    

    return signature


# runCreateSignature

In [45]:
def runCreateSignature(isAAorEUR):
    # runCreateSignature loads the relevant files and creates a robust signature via LASSO.
    # Inputs: dirName - the name of the directory of the training files
    #         isAAorEUR - train an AA signature (=0) or EUR (=1)
    # Output: signature - a cell array with the robust signature

    if isAAorEUR == 0:
        fileName = 'AA_train.txt'
    elif isAAorEUR == 1:
        fileName = 'EUR_train.txt'
    else:
        raise ValueError('isAAorEUR can take the values 0 or 1 only')

    NUMBER_OF_SUBSETS = 100
    SUBSET_CUTOFF = 0.5

    exprData, residuals, geneTissue = LoadData(fileName)

    allGenesSelected, features_sets = CreateLassoSignature(exprData, residuals, geneTissue, NUMBER_OF_SUBSETS)
#     print("All Genes Selected : ",allGenesSelected)
#     print("Feature Sets : ",features_sets)

    signature = SelectRobustSignature(features_sets, allGenesSelected[0], SUBSET_CUTOFF)

    return signature


# Signatures for AA Cohort

In [49]:
signaturesAA = runCreateSignature(0)
print(signaturesAA)

['Artery_CoronarySNAP25', 'Brain_Caudate_basal_gangliaNCOA1', 'Brain_CerebellumALOX5', 'Brain_CerebellumSTX4', 'Brain_CortexALOX5', 'Brain_Frontal_Cortex_BA9LRPAP1', 'Colon_TransverseLGALS2', 'Esophagus_MucosaSMAD3', 'Heart_Left_VentricleDAB1', 'LiverVKORC1', 'PancreasPROZ', 'PancreasPLCG2', 'SpleenPROZ', 'ThyroidTNFSF4', 'Whole_BloodAKT1', 'Artery_TibialLRPAP1', 'Esophagus_MucosaALOX5', 'Nerve_TibialLRPAP1', 'Adrenal_GlandSERPINF2']


# Signatures for EUR Cohort

In [50]:
signaturesEUR = runCreateSignature(1)
print(signaturesEUR)

['Adipose_SubcutaneousCRKL', 'Adipose_Visceral_OmentumGCLM', 'Adrenal_GlandCUBN', 'Adrenal_GlandCYP1A1', 'Artery_TibialPRKCB', 'Brain_Cerebellar_HemisphereGGCX', 'Brain_Cerebellar_HemisphereLRPAP1', 'Brain_CerebellumALOX5', 'Brain_CerebellumGGCX', 'Brain_Putamen_basal_gangliaSERPINF2', 'Colon_SigmoidTNFRSF1B', 'Esophagus_Gastroesophageal_JunctionPRKCB', 'Esophagus_MucosaF5', 'Esophagus_MuscularisARRB1', 'Heart_Atrial_AppendageITGA4', 'LiverVKORC1', 'PancreasPLCG2', 'PituitaryF2R', 'Skin_Not_Sun_Exposed_SuprapubicGGCX', 'TestisABCB1', 'TestisF2R', 'ThyroidLRP8', 'ThyroidVKORC1', 'Adrenal_GlandCTNNB1', 'Esophagus_MuscularisUBE2I', 'Muscle_SkeletalPSMA6', 'Whole_BloodLGALS2']


In [74]:
def find_first_capital(string):
    for i in range(len(string) - 1):
        if string[i].isupper() and (string[i + 1].isupper() or string[i + 1].isdigit()) :
            return i
    return -1

In [78]:
geneAA={}
for sig in signaturesAA:
    i=find_first_capital(sig)
    t=sig[i:]
    g=sig[:i]
    if t in geneAA:
        geneAA[t].append(g)
    else:
        geneAA[t]=[g]
print("Gene _ Tissue Signatures for AA Cohort")
print("--------------------------------------")
for key,value in geneAA.items():
    print(key,value)

Gene _ Tissue Signatures for AA Cohort
--------------------------------------
SNAP25 ['Artery_Coronary']
NCOA1 ['Brain_Caudate_basal_ganglia']
ALOX5 ['Brain_Cerebellum', 'Brain_Cortex', 'Esophagus_Mucosa']
STX4 ['Brain_Cerebellum']
BA9LRPAP1 ['Brain_Frontal_Cortex_']
LGALS2 ['Colon_Transverse']
SMAD3 ['Esophagus_Mucosa']
DAB1 ['Heart_Left_Ventricle']
VKORC1 ['Liver']
PROZ ['Pancreas', 'Spleen']
PLCG2 ['Pancreas']
TNFSF4 ['Thyroid']
AKT1 ['Whole_Blood']
LRPAP1 ['Artery_Tibial', 'Nerve_Tibial']
SERPINF2 ['Adrenal_Gland']


In [77]:
geneEUR={}
for sig in signaturesEUR:
    i=find_first_capital(sig)
    t=sig[i:]
    g=sig[:i]
    if t in geneEUR:
        geneEUR[t].append(g)
    else:
        geneEUR[t]=[g]
print("Gene _ Tissue Signatures for EUR Cohort")
print("---------------------------------------")
for key,value in geneEUR.items():
    print(key,value)

Gene _ Tissue Signatures for EUR Cohort
---------------------------------------
CRKL ['Adipose_Subcutaneous']
GCLM ['Adipose_Visceral_Omentum']
CUBN ['Adrenal_Gland']
CYP1A1 ['Adrenal_Gland']
PRKCB ['Artery_Tibial', 'Esophagus_Gastroesophageal_Junction']
GGCX ['Brain_Cerebellar_Hemisphere', 'Brain_Cerebellum', 'Skin_Not_Sun_Exposed_Suprapubic']
LRPAP1 ['Brain_Cerebellar_Hemisphere']
ALOX5 ['Brain_Cerebellum']
SERPINF2 ['Brain_Putamen_basal_ganglia']
TNFRSF1B ['Colon_Sigmoid']
F5 ['Esophagus_Mucosa']
ARRB1 ['Esophagus_Muscularis']
ITGA4 ['Heart_Atrial_Appendage']
VKORC1 ['Liver', 'Thyroid']
PLCG2 ['Pancreas']
F2R ['Pituitary', 'Testis']
ABCB1 ['Testis']
LRP8 ['Thyroid']
CTNNB1 ['Adrenal_Gland']
UBE2I ['Esophagus_Muscularis']
PSMA6 ['Muscle_Skeletal']
LGALS2 ['Whole_Blood']
